In [1]:
import urllib.request
from bs4 import BeautifulSoup
import os
from joblib import Parallel, delayed
from multiprocessing import Pool
from multiprocessing import Process
import multiprocessing
import time
import requests 
import urllib.request
import json
import csv  
from tqdm import tqdm
import PyPDF2 

from multiprocessing import Pool, freeze_support, RLock
from multiprocessing import current_process
from p_tqdm import p_map, p_umap, p_imap, p_uimap
from clint.textui import progress

In [2]:
def get_text_html(link):
    return BeautifulSoup(requests.get(url=link).text, 'html.parser')

In [3]:
def create_dir(path):
    if os.path.exists(path) == False:
        os.mkdir(path)

In [4]:
def get_pages_by_author(data):
    pages = {}
    name_author = ''
    for item in data:
        tag_name = item.name
        if tag_name == 'h2':
            name_author = item.text.replace(' ¶','')
#             print(name_author)
        elif tag_name == 'ul':
            name_book = item.text
            links_book = []
            for i in item.findChildren('li', recurdive=False):
                if 'English' in i.text:
                    links_on_book = i.find_all('a')
                    for link in links_on_book:
                        if 'ebooks' in link['href']:
                            links_book.append({f"https://www.gutenberg.org{link['href']}": link.text})
            if len(links_book) > 0:
                pages[name_author] = links_book
    return pages

In [5]:
def get_direct_reference(link):
    result = 'empty'
    page = get_text_html(link)
    data = page.find_all("td", {"class": "noscreen"})
    for i in data:
        inner_text = i.text
        if '.txt' in inner_text:
            result = inner_text
        elif '.pdf' in inner_text:
            result = inner_text
    return result

In [6]:
def download_by_link(link, path):
    if '.txt' in link:
        urllib.request.urlretrieve(link, path)
    elif '.pdf' in link:
        print('Working with pdf: ', link)
        temp_path = f"{path} temp.pdf"
        urllib.request.urlretrieve(link, temp_path)
        
        pdffileobj=open(temp_path,'rb')
        pdfReader = PyPDF2.PdfFileReader(pdffileobj)
        count_pages = pdfReader.numPages
        file1=open(f"{path}","a")
        for i in range(count_pages):
            file1.write(pdfReader.getPage(i).extractText())
        file1.close()
        os.remove(temp_path)

In [7]:
def download_by_item(item):
    path = item[0]
    name_author = item[1][0]
    create_dir(f"{path}/{name_author}")
    list_books = item[1][1]
    for page_info in list_books:
        for link in page_info.keys():
            direct_reference = get_direct_reference(link)
            download_by_link(direct_reference, f"{path}/{name_author}/{page_info[link][:250]}.txt")

In [8]:
def get_pairs_link_path(dict_info, dir_name):
    count = 1
    pairs_dir_name_book_info = []
    dict_info_items = list(dict_info.items())
    
    for item in dict_info_items:
#         if count > 1:
#             break
        pairs_dir_name_book_info.append((dir_name, item))
#        print(count)
        count += 1
    return pairs_dir_name_book_info

In [9]:
def download_files(link_path):
    download_by_link(link_path[0], link_path[1])

In [10]:
def save_html_page(link, path):
    page = get_text_html(link)
    create_dir(f"{path}")
    items = link.split('/')
    file=open(f"{path}/{items[len(items) - 1]}.html","a")
    file.write(str(page))
    file.close()

In [79]:
def save_html_pages(letter_author, name_author, path, books_info):
    create_dir(f"{path}/{letter_author}")
    new_path = f"{path}/{letter_author}/{name_author}"
    
    create_dir(new_path)
    for book_info in books_info:
        item = list(book_info.items())
        link = item[0][0]
        name_book = item[0][1]
        save_html_page(link, new_path)

In [80]:
def get_info_authors_by_letter(letter):
    url = f"https://www.gutenberg.org/browse/authors/{letter}"
    page = get_text_html(url)
    
#     return get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).find('div').children)
    return get_pages_by_author(page.find("div", {"class": "pgdbbyauthor"}).children)

In [81]:
def save_html(letter, info_authors):
    for name_author, i in zip(info_authors, tqdm(range(len(info_authors)))):
        save_html_pages(letter, name_author, f"..", info_authors[name_author])

In [82]:
def save_html_fast(letter, info_authors, path):
    size_pack = 8
    size = len(info_authors)
    count_pack = size // size_pack
    mod_size = count_pack * size_pack
    diff = size - mod_size
    keys = list(info_authors.keys())
    process_list = []
    for i, count_iter in zip(range(count_pack), tqdm(range(count_pack))):
        for j in range(size_pack):
            key = keys[j + i * size_pack]
            process_list.append(Process(target=save_html_pages, args=(letter, key, path, info_authors[key])))
        
        for t in process_list:
            t.start()
        
        for t in process_list:
            t.join()
            
        for t in process_list:
            t.close()
        process_list.clear()

    print(f"{diff} - document(s) remained.")
    for i in range(diff):
        print(i + 1, 'Name:', keys[i + mod_size])
        save_html_pages(letter, keys[i + mod_size], path, info_authors[keys[i + mod_size]])
    print(f"\nLetter: '{letter}' was download. Count of documents: {size}")

In [ ]:
def save_all_authors():
    alphabet = 'defghijklmnopqrsuvwxyz'
    path = '../Test'
    create_dir(path)
    for letter in alphabet:
        save_html_fast(f"{letter.capitalize()}", get_info_authors_by_letter(letter), path)

In [ ]:
save_all_authors()

In [152]:
def get_dictionary(tags):
    name = ''
    result = {}
    for tag in tags:
        if tag.name == 'th':
            name = tag.text
        elif tag.name == 'td' and name != '':
            result = {name: tag.text.rstrip().lstrip()}
    return result

In [ ]:
isAuthor = 'Author' in dictionary
            isTitle = 'Title' in dictionary

In [220]:
def get_list_dictionary(link):
    list_info = []
    page = get_text_html(link)
    data = page.find("table", {"class": "bibrec"}).find_all('tr')
    for tr in data:
        dictionary = get_dictionary(tr)
#         print(dictionary)
        if len(dictionary) > 0:
            res = 'Downloads' in dictionary
            if res == False:
                list_info.append(dictionary)
    return list_info

In [221]:
def delete_duplicate(list_dict):
    keys = []
    for item in list_dict:
        keys.append(list(item.keys())[0])
    unique_keys = set(keys)
    new_dict = {}
    for key in unique_keys:
        new_list = []
        for item in list_dict:
            #print(key, item, key in item)
            if key in item:
                new_list.append(item[key])
        new_dict[key] = new_list
    return new_dict

In [238]:
def get_raw_data(pages):
    print(len(pages))
    list_dictionaty_without_duplicate = []
    count = 0
    for key in pages.keys():
        print(count, key)
        for item in pages[key]:
            list_dictionary = get_list_dictionary(item)
            list_dictionaty_without_duplicate.append(delete_duplicate(list_dictionary))
        count += 1
        if count > 50:
            break
    return list_dictionaty_without_duplicate

In [ ]:
raw_data = get_raw_data(pages)

In [244]:
def get_json(dictionary):
    return json.dumps(dictionary, indent = 0)

In [273]:
def create_csv(name, data):
    header = ['Author', 'Title']
    with open(name, 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for i in data:
            if 'Author' in i:
                for author in i['Author']:
                    for title in i['Title']:
                        writer.writerow([author, title])
            if 'Contributor' in i:
                for contributor in i['Contributor']:
                    for title in i['Title']:
                        writer.writerow([contributor, title])

create_csv('parsed_data.csv', raw_data)